In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
%run experiment.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from mlutils.mlflow.utils import create_runs_for_params, get_unfinished_run_ids, get_runs, get_unfinished_runs, get_run_params, experiment_name_to_id
from glob import glob
from mlutils.distributedcomputing.experiments import run_experiments
from sklearn.model_selection import ParameterGrid
from mlflow import MlflowClient
import mlflow



In [4]:
all_processed_data_files = glob(f"../datasets/notebooks/processed/*.csv")

all_file_names = [
    file.split('/')[-1].split('train')[0].rstrip('-') for file in all_processed_data_files if 'tra.csv' in file
]
print(all_file_names[:5])

['shuttle-5-5tra.csv', 'pima-5-5tra.csv', 'wisconsin-5-4tra.csv', 'automobile-5-2tra.csv', 'spambase-5-1tra.csv']


In [5]:
train_file_paths = [it for it in all_processed_data_files if 'tra.csv' in it]
dataset_names = {path.split('/')[-1].split('-')[0] for path in train_file_paths}
selected_datasets = []
for name in dataset_names:
    selected_datasets.append([p for p in train_file_paths if name in p][0])

In [9]:
len(train_file_paths)

100

In [12]:
selected_datasets

['../datasets/notebooks/processed/heart-5-5tra.csv',
 '../datasets/notebooks/processed/ecoli-5-3tra.csv',
 '../datasets/notebooks/processed/mushroom-5-3tra.csv',
 '../datasets/notebooks/processed/haberman-5-3tra.csv',
 '../datasets/notebooks/processed/titanic-5-4tra.csv',
 '../datasets/notebooks/processed/shuttle-5-5tra.csv',
 '../datasets/notebooks/processed/wine-5-2tra.csv',
 '../datasets/notebooks/processed/nursery-5-2tra.csv',
 '../datasets/notebooks/processed/vowel-5-5tra.csv',
 '../datasets/notebooks/processed/balance-5-5tra.csv',
 '../datasets/notebooks/processed/car-5-5tra.csv',
 '../datasets/notebooks/processed/breast-5-2tra.csv',
 '../datasets/notebooks/processed/banana-5-4tra.csv',
 '../datasets/notebooks/processed/mammographic-5-5tra.csv',
 '../datasets/notebooks/processed/chess-5-5tra.csv',
 '../datasets/notebooks/processed/pima-5-5tra.csv',
 '../datasets/notebooks/processed/automobile-5-2tra.csv',
 '../datasets/notebooks/processed/wisconsin-5-4tra.csv',
 '../datasets/note

In [7]:
mlflow.set_tracking_uri("sqlite:///experiments.db")

In [14]:
params_v6 = ParameterGrid({
    'train_path': selected_datasets,
    'base_clf': ["perceptron", "dt", "knn", "nb"],
    'min_samples': [10, 25],
    "complexity_measure": ["f2", "n3", "l3", "density"],
    "min_split_percentage": [0.1, 0.3, 0.5]
})

In [15]:
base_params = ParameterGrid({
    'train_path': train_file_paths,
})

In [8]:
client = MlflowClient(tracking_uri="sqlite:///experiments.db")

In [ ]:
base_runs = create_runs_for_params(base_params, experiment_name='base', client=client)


In [18]:
v6_runs = create_runs_for_params(params_v6, experiment_name='v6', client=client)

2023-08-22 08:40:45.191 | INFO     | mlutils.mlflow.utils:create_runs_for_params:32 - Experiment not found, creating new. Id=4
100%|██████████| 1920/1920 [00:18<00:00, 102.99it/s]


In [13]:
v6_runs = get_runs(experiment_name_to_id("v6", client=client))
base_runs = get_runs(experiment_name_to_id("base", client=client))

In [ ]:
v1_run_ids = v1_runs.run_id

In [30]:
unfinished_runs = get_unfinished_run_ids(experiment_name_to_id('v6', client=client))

In [31]:
len(unfinished_runs)

785

In [ ]:
run_experiments(unfinished_runs, 'do_experiment_v1', notebook_path="experiment.ipynb", backend='papermill', n_jobs=5)

2023-08-24 17:55:05.265 | INFO     | mlutils.distributedcomputing.experiments:run_experiments:11 - {'notebook_path': 'experiment.ipynb', 'n_jobs': 5}
2023-08-24 17:55:05.268 | WARNING  | mlutils.distributedcomputing.experiments:run_experiments:21 - Using Papermill backend. Make sure you execute this function not from experiment notebook!
2023-08-24 17:55:05.270 | INFO     | mlutils.distributedcomputing.job_scheduling:create_experiment_notebook:91 - 
    Creating experiment notebook
        notebook_name=experiment.ipynb
        experiment_function=do_experiment_v1
        instance_id_param_name=EXPERIMENT_INSTANCE_ID
        
2023-08-24 17:55:05.275 | INFO     | mlutils.distributedcomputing.experiments:run_experiments:34 - Notebook path=/Users/bgulowaty/studia/projekty/quad-splits/experiment-014077fa-662c-4ad5-94a3-9fae35f278b5.ipynb
2023-08-24 17:55:05.282 | INFO     | mlutils.distributedcomputing.job_scheduling:run_in_papermill:37 - 
    papermill_path=/Users/bgulowaty/studia/projekt

In [ ]:
v4_runs.run_id[25]

In [15]:
run_experiments(base_runs.run_id, do_experiment_base, backend='plain')

2023-08-23 23:15:53.371 | INFO     | mlutils.distributedcomputing.experiments:run_experiments:11 - {}
2023-08-23 23:15:53.382 | INFO     | __main__:do_experiment_base:5 - {'train_path': '../datasets/notebooks/processed/banana-5-3tra.csv'}
2023-08-23 23:15:53.774 | INFO     | __main__:do_experiment_base:5 - {'train_path': '../datasets/notebooks/processed/vowel-5-2tra.csv'}
2023-08-23 23:15:53.938 | INFO     | __main__:do_experiment_base:5 - {'train_path': '../datasets/notebooks/processed/nursery-5-5tra.csv'}
2023-08-23 23:15:54.350 | INFO     | __main__:do_experiment_base:5 - {'train_path': '../datasets/notebooks/processed/car-5-2tra.csv'}
2023-08-23 23:15:54.453 | INFO     | __main__:do_experiment_base:5 - {'train_path': '../datasets/notebooks/processed/yeast-5-2tra.csv'}
2023-08-23 23:15:54.607 | INFO     | __main__:do_experiment_base:5 - {'train_path': '../datasets/notebooks/processed/wisconsin-5-1tra.csv'}
2023-08-23 23:15:54.683 | INFO     | __main__:do_experiment_base:5 - {'train_

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [13]:
v4_runs.status.value_counts()

status
RUNNING     8832
FAILED       672
FINISHED      96
Name: count, dtype: int64